In [58]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

In [59]:
train_path = '../../data/train.csv'
test_path  = '../../data/test.csv'
gyotongbu_price_path = '../../data/gyotongbu_actual_transaction_price.csv'
dt = pd.read_csv(train_path)
dt_test = pd.read_csv(test_path)
dt_gyotongbu_price = pd.read_csv(gyotongbu_price_path)

In [60]:
dt['is_test'] = 0
dt_test['is_test'] = 1
concat = pd.concat([dt, dt_test])     # 하나의 데이터로 만들어줍니다.

In [61]:
concat = concat.rename(columns={'전용면적(㎡)':'전용면적'})

In [62]:
concat['등기신청일자'] = concat['등기신청일자'].replace(' ', np.nan)
concat['거래유형'] = concat['거래유형'].replace('-', np.nan)
concat['중개사소재지'] = concat['중개사소재지'].replace('-', np.nan)

In [63]:
concat["아파트정보"] = concat["시군구"] + " " + " " + concat["전용면적"].astype(str)\
      + " " + concat["계약년월"].astype(str) + concat["계약일"].astype(str) + " " + concat["층"].astype(str) + "층"

In [64]:
dt_gyotongbu_price["아파트정보"] = dt_gyotongbu_price["시군구"] + " " + dt_gyotongbu_price["전용면적(㎡)"].astype(str)\
      + " " + dt_gyotongbu_price["계약년월"].astype(str) + dt_gyotongbu_price["계약일"].astype(str) + " " + dt_gyotongbu_price["층"].astype(str) + "층"

In [65]:
concat["아파트정보"] = concat["아파트정보"].str.strip()
dt_gyotongbu_price["아파트정보"] = dt_gyotongbu_price["아파트정보"].str.strip()

In [66]:
def del_comma(string):
    import re
    return re.sub(',', '', string)

In [67]:
dt_gyotongbu_price["거래금액(만원)"] = dt_gyotongbu_price["거래금액(만원)"].apply(del_comma).astype(float)

In [68]:
concat.sort_values(by="아파트정보", ascending=False, inplace=True)
dt_gyotongbu_price.sort_values(by="아파트정보", ascending=False, inplace=True)

In [69]:
dt_gyotongbu_price[dt_gyotongbu_price["아파트정보"]=="서울특별시 중구 황학동 82.0 2017041 2층"]

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,아파트정보
816303,서울특별시 중구 황학동,1544,1544.0,0.0,(1544-0),82.0,201704,1,35000.0,2,2003,난계로15길 36-8,NaN,,-,-,서울특별시 중구 황학동 82.0 2017041 2층


In [51]:
dt_gyotongbu_price = dt_gyotongbu_price[["아파트정보", "거래금액(만원)"]]

In [55]:
dt_gyotongbu_price

,아파트정보,거래금액(만원)
1103970,서울특별시 중랑구 중화동 96.4 20210130 11층,89000.0
1060451,서울특별시 중랑구 중화동 96.4 2020081 5층,73000.0
976332,서울특별시 중랑구 중화동 96.4 20190427 14층,56500.0
901196,서울특별시 중랑구 중화동 96.4 20180331 13층,53500.0
901195,서울특별시 중랑구 중화동 96.4 20180327 10층,54000.0
...,...,...
331004,서울특별시 강남구 개포동 101.0 2013092 5층,55000.0
510,서울특별시 강남구 개포동 101.0 2007029 5층,51000.0
976877,서울특별시 강남구 개포동 100.5 20200610 5층,77500.0
605459,서울특별시 강남구 개포동 100.5 2016064 5층,52000.0


In [53]:
merge = pd.merge(concat, dt_gyotongbu_price, on="아파트정보", how='left')
merge

,시군구,번지,본번,부번,아파트명,전용면적,계약년월,계약일,층,건축년도,...,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,is_test,아파트정보,거래금액(만원)
0,서울특별시 중랑구 중화동,454,454.0,0.0,동양엔파트2차,96.4,202101,30,11,2013,...,NaN,NaN,NaN,NaN,NaN,NaN,89000.0,0,서울특별시 중랑구 중화동 96.4 20210130 11층,NaN
1,서울특별시 중랑구 중화동,454,454.0,0.0,동양엔파트2차,96.4,202008,1,5,2013,...,NaN,NaN,NaN,NaN,NaN,NaN,73000.0,0,서울특별시 중랑구 중화동 96.4 2020081 5층,NaN
2,서울특별시 중랑구 중화동,454,454.0,0.0,동양엔파트2차,96.4,201904,27,14,2013,...,NaN,NaN,NaN,NaN,NaN,NaN,56500.0,0,서울특별시 중랑구 중화동 96.4 20190427 14층,NaN
3,서울특별시 중랑구 중화동,454,454.0,0.0,동양엔파트2차,96.4,201803,31,13,2013,...,NaN,NaN,NaN,NaN,NaN,NaN,53500.0,0,서울특별시 중랑구 중화동 96.4 20180331 13층,NaN
4,서울특별시 중랑구 중화동,454,454.0,0.0,동양엔파트2차,96.4,201803,27,10,2013,...,NaN,NaN,NaN,NaN,NaN,NaN,54000.0,0,서울특별시 중랑구 중화동 96.4 20180327 10층,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,서울특별시 강남구 개포동,1260-11,1260.0,11.0,우찬현대,101.0,201309,2,5,2006,...,NaN,NaN,NaN,NaN,NaN,NaN,55000.0,0,서울특별시 강남구 개포동 101.0 2013092 5층,NaN
1128090,서울특별시 강남구 개포동,1260-11,1260.0,11.0,우찬현대,101.0,200702,9,5,2006,...,NaN,NaN,NaN,NaN,NaN,NaN,51000.0,0,서울특별시 강남구 개포동 101.0 2007029 5층,NaN
1128091,서울특별시 강남구 개포동,1260-11,1260.0,11.0,우찬현대,100.5,202006,10,5,2006,...,NaN,NaN,NaN,NaN,NaN,NaN,77500.0,0,서울특별시 강남구 개포동 100.5 20200610 5층,NaN
1128092,서울특별시 강남구 개포동,1260-11,1260.0,11.0,우찬현대,100.5,201606,4,5,2006,...,NaN,NaN,NaN,NaN,NaN,NaN,52000.0,0,서울특별시 강남구 개포동 100.5 2016064 5층,NaN


In [19]:
merge["거래금액(만원)"] = merge["거래금액(만원)"].apply(del_comma).astype(float)
merge["거래금액(만원)"]

Series([], Name: 거래금액(만원), dtype: float64)

In [20]:
merge[(merge["거래금액(만원)"]!=merge["target"])].head(50)

,시군구_x,번지_x,본번_x,부번_x,아파트명,전용면적,계약년월_x,계약일_x,층_x,건축년도_x,...,계약년월_y,계약일_y,거래금액(만원),층_y,건축년도_y,도로명_y,해제사유발생일_y,등기신청일자_y,거래유형_y,중개사소재지_y


In [21]:
merge["아파트명"].unique()

array([], dtype=object)

In [22]:
pd.read_csv(gyotongbu_price_path).columns

Index(['시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', '계약년월', '계약일', '거래금액(만원)',
       '층', '건축년도', '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지'],
      dtype='object')

## 국토교통부 실거래가 데이터는 쓸모없다

- 사실상 실거래가는 같음
- 여기서 얻을 수 있는건 test의 target값 뿐...
- 다른 NaN 값도 채울 수 없음